In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder

In [2]:
# Cargar dataset de prueba
data_test = pd.read_csv('../data/raw/test.csv')

#data_test = pd.read_csv('test.csv')

In [3]:
data_test.head()

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating
0,1950,67,Female,87167,Weekly,152.136795,Chinese,Lunch,4,Celebration,Dine-in,0,1,1,14.722728,5,5,4
1,1145,65,Female,45416,Weekly,119.837286,Italian,Breakfast,1,Celebration,Takeaway,1,0,0,8.717860,2,3,5
2,960,54,Female,48233,Weekly,31.424327,American,Lunch,5,Casual,Dine-in,0,0,0,19.326013,5,2,2
3,1057,61,Female,69799,Weekly,67.879151,American,Dinner,5,Celebration,Dine-in,0,1,1,24.708839,3,4,2
4,2030,52,Female,108514,Monthly,155.891539,Mexican,Lunch,6,Celebration,Dine-in,0,1,1,35.944814,4,3,5


Proceso de preparación de datos para predecir

In [4]:
# Cargar configuraciones de ingeniería de características
with open('../artifacts/feature_eng_configs.pkl', 'rb') as f:
    feature_eng_configs = pickle.load(f)

#with open('feature_eng_configs.pkl', 'rb') as f:
    #feature_eng_configs = pickle.load(f)

In [5]:
feature_eng_configs

{'visit_freq_counts': VisitFrequency
 Weekly     906
 Monthly    411
 Rarely     306
 Daily      195
 Name: count, dtype: int64,
 'dining_occ_counts': DiningOccasion
 Celebration    728
 Casual         545
 Business       545
 Name: count, dtype: int64}

In [6]:
data_test.drop(["CustomerID", "PreferredCuisine", "Gender", "TimeOfVisit", "AverageSpend", "Age"], axis=1, inplace=True)

# codificación de variable MealType
data_test['MealType'] = pd.get_dummies(data_test['MealType'], drop_first=True).astype(int)

# codificación de Variables VisitFrequency y DiningOccasion
data_test['VisitFrequency'] = data_test['VisitFrequency'].map(feature_eng_configs['visit_freq_counts'])
data_test['DiningOccasion'] = data_test['DiningOccasion'].map(feature_eng_configs['dining_occ_counts'])

data_test.head()

,Income,VisitFrequency,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating
0,87167,906,4,728,0,0,1,1,14.722728,5,5,4
1,45416,906,1,728,1,1,0,0,8.717860,2,3,5
2,48233,906,5,545,0,0,0,0,19.326013,5,2,2
3,69799,906,5,728,0,0,1,1,24.708839,3,4,2
4,108514,411,6,728,0,0,1,1,35.944814,4,3,5


-----------------

--------------------

In [7]:
data_test.isnull().mean()

Income                  0.0
VisitFrequency          0.0
GroupSize               0.0
DiningOccasion          0.0
MealType                0.0
OnlineReservation       0.0
DeliveryOrder           0.0
LoyaltyProgramMember    0.0
WaitTime                0.0
ServiceRating           0.0
FoodRating              0.0
AmbianceRating          0.0
dtype: float64

In [8]:
# Cargar el scaler estándar si fue usado previamente
with open('../artifacts/std_scaler.pkl', 'rb') as f:
    std_scaler = pickle.load(f)

#with open('std_scaler.pkl', 'rb') as f:
    #std_scaler = pickle.load(f)

In [10]:
# Cargar el modelo guardado
with open('../models/random_forest_v1.pkl', 'rb') as f:
    modelo = pickle.load(f)

#with open('random_forest_v1.pkl', 'rb') as f:
    #modelo = pickle.load(f)

In [11]:
# Estandarizar las variables del dataset de prueba
X_data_test_std = std_scaler.transform(data_test)


In [12]:
# Realizar predicciones
model_predicts = modelo.predict(X_data_test_std)

# Mostrar las predicciones
print("Predicciones del modelo:")
print(model_predicts)

Predicciones del modelo:
[1 0 0 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 0 0 1
 1 0 1 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1 0 0 1 0
 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 1 1
 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1
 1 0 1 0 1 1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 1 0 1
 1 1 1 0 1 1 0 1 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0
 0 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0
 0 1 1 1 1 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 1 1 1 1
 1 0 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1
 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1 1 0 0
 1 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 0
